Exercise 3

Select all the frequent aspects using Association Rule Mining algorithm with minimum 0.5% and 1% support. (Submit the top-10 most frequent aspects in decreasing order of support.). What difference do you see in using 0.5% vs 1% minimum support? 

Note: You need to consider each sentence as a transaction and not the whole review. Same feature might be mentioned multiple number of times in a single review. As a result, the count of frequent features may be more than the total number of reviews. 

If two or more nouns are occurring consecutively, then you should treat it as follows: for a bigram “battery life”, pass three nouns to the association rule mining algorithm: “battery”, “life” and “battery life”. With this approach if “battery life” is a frequently occurring phrase then that will remain after association rule mining whereas less frequently occurring phrases such as “battery duration” will only contribute to the feature “battery”

In [1]:
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import ngrams
from fp_growth import find_frequent_itemsets
import subprocess
from collections import Counter
import re
from itertools import chain, combinations
import collections

In [2]:
file_content=''
with open('data\\CanonG3_untagged.txt','r') as unTag:
    file_content = unTag.read()
    sentences = sent_tokenize(file_content)
    #for tkn in tokens:
    #    print(tkn)
len(sentences)

597

In [3]:
tag_nouns=[]
with open('Output\\aspects.csv','r') as aspects:
    for asp in aspects.readlines():
        tag_nouns.append(asp.rstrip('\n').split(','))

In [4]:
tag_nouns[0],len(tag_nouns)

(['picture', 'quality', 'color', 'canon', 'powershot', 'g3', 'purchase'], 562)

In [5]:
#sentences that do not contain any noun tags at all are as follows
len(sentences)-len(tag_nouns)

35

In [6]:
def BiAndTriGramGeneForTransactionAspects(trans_Nouns):
    transaction_comb_nouns=[]
    allCombinations = chain(*map(lambda x: combinations(trans_Nouns, x), range(1, len(trans_Nouns)+1)))
    for subset in allCombinations:
        if(len(subset)<4):
            wordComb=' '.join(subset)
            #add to valid transaction combinations only if it is present in CanonG3_untagged.txt
            if wordComb in file_content:
                transaction_comb_nouns.append(' '.join(subset))
    return transaction_comb_nouns

In [7]:
#Example to generate combinations
BiAndTriGramGeneForTransactionAspects(['picture', 'quality', 'color', 'canon', 'powershot', 'g3', 'purchase','purchase'])


['picture',
 'quality',
 'color',
 'canon',
 'powershot',
 'g3',
 'purchase',
 'purchase',
 'picture quality',
 'canon powershot',
 'canon g3',
 'powershot g3',
 'canon powershot g3']

In [8]:
ptr=0
while ptr< len(tag_nouns):
    tag_nouns[ptr] = BiAndTriGramGeneForTransactionAspects(tag_nouns[ptr])
    ptr+=1

In [9]:
tag_nouns[0]

['picture',
 'quality',
 'color',
 'canon',
 'powershot',
 'g3',
 'purchase',
 'picture quality',
 'canon powershot',
 'canon g3',
 'powershot g3',
 'canon powershot g3']

In [10]:
aspects_with_N_Grams='3_aspects_with_N_Grams.csv'

In [11]:
with open('Output\\'+aspects_with_N_Grams,'w') as aspNG:
    for trans_nouns in tag_nouns:
        aspNG.write(','.join(trans_nouns)+'\n')

In [12]:
#python -m fp_growth -s 3 examples/aspects.csv
with open("Output\\3_ARM_FP_GROWTH_0_5_percent_support.txt", "w") as fpOut:
    subprocess.call(["python","-m", "fp_growth", "-s", "3", "Output\\3_aspects_with_N_Grams.csv"], stdout=fpOut)

In [13]:
with open("Output\\3_ARM_FP_GROWTH_1_percent_support.txt", "w") as fpOut:
    subprocess.call(["python","-m", "fp_growth", "-s", "6", "Output\\3_aspects_with_N_Grams.csv"], stdout=fpOut)

Now Calculate Top 10 Freq Aspects

In [14]:
def GenerateTopNFrequentAspects(fpInpFile,N):
    with open('Output\\'+fpInpFile,'r') as inpfile:
        freq_Words = []
        c = collections.Counter()
        for line in inpfile.readlines():
            freq_Words.append(re.findall(r"['\"](.*?)['\"]", line))
            fp_sup= line.strip('[').strip('\n').split('] ')
            if(len(fp_sup)==2):
                c[str(fp_sup[0])]=str(fp_sup[1])
    #print top 10 requent aspects in decreasing order of support
    top_10_freq_asp_dec_order_sup='Output\\3_Top_10_freq_asp_dec_order_sup'+fpInpFile
    with open(top_10_freq_asp_dec_order_sup,'w') as fpIn:
        f_cont=str(c.most_common(N)).replace('),','\n').replace('[','').replace(')]','').replace('(','').replace("\"'","['").replace('",',']').replace(" [","[")
        fpIn.write(f_cont)
    
    print('***************************')
    print('File for Top ' + str(N) + 'freq aspects with decreasing order of support here: ' + top_10_freq_asp_dec_order_sup)
    print('***************************')
    print('print top '+str(N)+' requent aspects in decreasing order of support')
    print(f_cont)
    
    aspects=[]
    for transaction in freq_Words:
        for aspect in transaction:
            aspects.append(aspect)
    counts = Counter(aspects)
    #print all frequent aspects
    allFreqAspects='Output\\3_All_Frequent_Aspects'+fpInpFile
    with open(allFreqAspects,'w') as allFreqAsp:
        allFreqAsp.write(str(counts).replace('Counter({','').replace(',','\n').replace('})','').replace(" '","'"))
        
    #now print top N frequent aspects
    outputfileName='Output\\3_Top_10_Frequent_Aspects'+fpInpFile
    with open(outputfileName,'w') as freqAsp:
        f_cont_asp=str(counts.most_common(N)).replace('(','').replace('),','\n').replace('[','').replace(')]','').replace(" '","'")
        freqAsp.write(f_cont_asp)
    
    print('***************************')
    print('Output Generated in file: '+outputfileName)
    print(f_cont_asp)
    print('***************************')  
    

In [15]:
#Generate TOP 10 Freq Aspects for 0.5% Min Support
GenerateTopNFrequentAspects('3_ARM_FP_GROWTH_0_5_percent_support.txt',10)

***************************
File for Top 10freq aspects with decreasing order of support here: Output\3_Top_10_freq_asp_dec_order_sup3_ARM_FP_GROWTH_0_5_percent_support.txt
***************************
print top 10 requent aspects in decreasing order of support
['lens', 'lens cap', 'cap', 'camera cap'] '98'
['camera', 'g3', 'battery', 'g3 camera', 'canon camera'] '97'
['camera', 'lens cap', 'cap', 'camera cap'] '97'
['camera', 'canon', 'battery', 'canon g3', 'canon camera', 'life', 'battery life', 'canon g3 camera'] '96'
['camera', 'canon', 'battery', 'canon g3', 'g3 camera', 'canon camera', 'life', 'battery life'] '96'
['camera', 'canon', 'battery', 'canon g3', 'g3 camera', 'canon camera', 'megapixel', 'battery life', 'canon g3 camera'] '96'
['camera', 'canon', 'battery', 'canon g3', 'g3 camera', 'canon camera', 'megapixel', 'canon g3 camera'] '96'
['camera', 'canon', 'battery', 'canon g3', 'g3 camera', 'canon camera', 'megapixel', 'life', 'battery life', 'canon g3 camera'] '96'
['came

In [16]:
#Generate TOP 10 Freq Aspects for 1% Min Support
GenerateTopNFrequentAspects('3_ARM_FP_GROWTH_1_percent_support.txt',10)

***************************
File for Top 10freq aspects with decreasing order of support here: Output\3_Top_10_freq_asp_dec_order_sup3_ARM_FP_GROWTH_1_percent_support.txt
***************************
print top 10 requent aspects in decreasing order of support
['camera', 'g3', 'battery', 'g3 camera', 'canon camera'] '97'
['camera', 'canon', 'battery', 'canon g3', 'g3 camera', 'canon camera', 'life', 'battery life'] '96'
['camera', 'canon', 'battery', 'canon g3', 'g3 camera', 'g2', 'megapixel', 'battery life', 'canon g3 camera', 'megapixel camera'] '96'
['camera', 'canon', 'battery', 'canon g3', 'g3 camera', 'g2', 'megapixel', 'battery life', 'way', 'canon g3 camera', 'megapixel camera'] '96'
['camera', 'canon', 'battery', 'canon g3', 'g3 camera', 'g2', 'megapixel', 'canon g3 camera', 'megapixel camera'] '96'
['camera', 'canon', 'battery', 'canon g3', 'g3 camera', 'g2', 'megapixel', 'life', 'battery life', 'canon g3 camera', 'megapixel camera'] '96'
['camera', 'canon', 'battery', 'canon g